In [3]:
from pomocnicze_funkcje_sql import *
from scraper import parsuj_misje_z_url
from scraper import pobierz_soup, pobierz_tresc
from scraper_pomocnicze import wyscrapuj_linki_z_kategorii_z_paginacja
import time
from sqlalchemy import text

In [2]:
# czerwony_przycisk(utworz_engine_do_db())

In [5]:
kategorie = [
    "https://warcraft.wiki.gg/wiki/Category:Quests_at_80-83",
    "https://warcraft.wiki.gg/wiki/Category:Quests_at_80-90",
    "https://warcraft.wiki.gg/wiki/Category:Quests_at_83",
    "https://warcraft.wiki.gg/wiki/Category:Quests_at_83-88",
    "https://warcraft.wiki.gg/wiki/Category:Quests_at_88-90",
]

silnik = utworz_engine_do_db()

for kat_i, kat_url in enumerate(kategorie, start=1):
    print(f"\n=== KATEGORIA [{kat_i}/{len(kategorie)}]: {kat_url} ===")

    questy = wyscrapuj_linki_z_kategorii_z_paginacja(
        kategoria_url=kat_url,
        sleep_s=3
    )

    print(f"Znaleziono {len(questy)} questów")

    for i, url in enumerate(questy, start=1):
        print(f"  [{i}/{len(questy)}] Hashuję: {url}")

        try:
            wynik = parsuj_misje_z_url(url)

            misja_url = wynik.get("Źródło", {}).get("url")

            q = text("""
                SELECT MISJA_ID_MOJE_PK
                FROM dbo.MISJE
                WHERE MISJA_URL_WIKI = :url
            """)

            with silnik.connect() as conn:
                row = conn.execute(q, {"url": misja_url}).first()

            if not row:
                print("    → brak w MISJE, dodaję do LINKI_DO_SCRAPOWANIA")
                zapisz_link_do_scrapowania(
                    silnik=silnik,
                    url=misja_url,
                    zrodlo="wiki"
                )
                continue


            misja_id = row[0]

            zapisz_zrodlo_do_db(
                silnik=silnik,
                tabela_zrodlo="dbo.ZRODLO",
                misja_id=misja_id,
                wynik=wynik,
                zrodlo="wiki"
            )

            print("    + zapisano hashe do ZRODLO")

        except Exception as e:
            print(f"    ! BŁĄD: {e}")

        time.sleep(3)


=== KATEGORIA [1/5]: https://warcraft.wiki.gg/wiki/Category:Quests_at_80-83 ===
    - Strona kategorii #1: https://warcraft.wiki.gg/wiki/Category:Quests_at_80-83
Znaleziono 132 questów
  [1/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Favor_for_the_Lion
    + zapisano hashe do ZRODLO
  [2/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Fish!
    + zapisano hashe do ZRODLO
  [3/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Foe_Unseen
    + zapisano hashe do ZRODLO
  [4/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Grave_Concern
    → brak w MISJE, dodaję do LINKI_DO_SCRAPOWANIA
  [5/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Hungry_Flock
    + zapisano hashe do ZRODLO
  [6/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Path_Not_Yet_Chosen
    + zapisano hashe do ZRODLO
  [7/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Ranger%27s_Spirit
    + zapisano hashe do ZRODLO
  [8/132] Hashuję: https://warcraft.wiki.gg/wiki/A_Real_Assignment
    + zapisano hashe do ZRODLO
  [9/132] Hashuję: ht

In [ ]:
# OK ===== to zwraca id misji, ktore mialy zmiane tresci
# OK ===== ponizszy kod uruchomic w petli ze zmiennymi pod inne nazwy hashy
# OK ===== nastepnie znalezc linki pod te misje, ktore zmienily tresc
# OK ===== usunac z bazy danych wszystkie powiazane elementy dla tych misji
# wyscrapowac dla nich na nowo tresc


roznice_hashe_usun_rekordy_z_db(silnik=utworz_engine_do_db())

▶ Szukam różnic w hashach...
  • HTML_HASH_GLOWNY_CEL: 1 misji
  • HTML_HASH_PODRZEDNY_CEL: 0 misji
  • HTML_HASH_TRESC: 0 misji
  • HTML_HASH_POSTEP: 0 misji
  • HTML_HASH_ZAKONCZENIE: 0 misji
  • HTML_HASH_NAGRODY: 0 misji
  • HTML_HASH_DYMKI: 0 misji
  • HTML_HASH_GOSSIP: 0 misji

▶ Łącznie misji do czyszczenia: 1
▶ ID misji: [1]

=== Czyszczę MISJA_ID = 1 ===
    - dbo.DIALOGI_STATUSY: usunięto 4 wierszy
    - dbo.MISJE_STATUSY: usunięto 8 wierszy
    - dbo.ZRODLO: usunięto 5 wierszy
    - dbo.MISJE: usunięto 1 wierszy
=== Zakończono MISJA_ID = 1 ===



In [ ]:
# =========== ODCZYTUJE LINKI Z KATEGORIE I PRZERZUCA OD RAZU WSZYSTKIE DO LINKI_DO_SCRAPOWANIA ===========
# =========== UŻYWAĆ W OSTATECZNOŚCI ===========
# kategorie = [
#     "https://warcraft.wiki.gg/wiki/Category:Quests_at_80-83",
#     "https://warcraft.wiki.gg/wiki/Category:Quests_at_80-90",
#     "https://warcraft.wiki.gg/wiki/Category:Quests_at_83",
#     "https://warcraft.wiki.gg/wiki/Category:Quests_at_83-88",
#     "https://warcraft.wiki.gg/wiki/Category:Quests_at_88-90",
# ]

# wyscrapuj_kategorie_questow_i_zapisz_linki_do_db(
#     silnik=silnik,
#     kategorie_urls=kategorie,
#     zrodlo="wiki",
#     sleep_s=3,
#     printuj_paginacje=True
# )


In [7]:
# =========== SCRAPUJE TYLKO TE MISJE/WYPOWIEDZI, KTORYCH NIE MA JESZCZE W BAZIE DANYCH ===========

linki = pobierz_linki_do_scrapowania(silnik)

print(f"Do przerobienia: {len(linki)} misji")

for i, url in enumerate(linki, start=1):
    print(f"\n[{i}/{len(linki)}] Przetwarzam: {url}")

    try:
        wynik = parsuj_misje_z_url(url)

        zapisz_npc_i_status_do_db_z_wyniku(
            silnik=silnik,
            tabela_npc="dbo.NPC",
            tabela_npc_statusy="dbo.NPC_STATUSY",
            szukaj_wg=["Start_NPC", "Koniec_NPC"],
            wyscrapowana_tresc=wynik,
            zrodlo="wiki",
            status="0_ORYGINAŁ"
        )

        misja_id = zapisz_misje_i_statusy_do_db_z_wyniku(
            silnik=silnik,
            wynik=wynik,
            tabela_npc="dbo.NPC",
            tabela_misje="dbo.MISJE",
            tabela_misje_statusy="dbo.MISJE_STATUSY",
            status="0_ORYGINAŁ"
        )

        zapisz_dialogi_statusy_do_db_z_wyniku(
            silnik=silnik,
            wynik=wynik,
            misja_id=misja_id,
            tabela_npc="dbo.NPC",
            tabela_npc_statusy="dbo.NPC_STATUSY",
            tabela_dialogi_statusy="dbo.DIALOGI_STATUSY",
            zrodlo="wiki",
            status="0_ORYGINAŁ"
        )

        zapisz_zrodlo_do_db(
            silnik=silnik,
            tabela_zrodlo="dbo.ZRODLO",
            misja_id=misja_id,
            wynik=wynik,
            zrodlo="wiki"
        )

        print(f"OK - zapisano MISJA_ID = {misja_id}")
        usun_link_z_kolejki(silnik, url)

    except Exception as e:
        print(f"BŁĄD przy {url}: {e}")

    time.sleep(3)

Do przerobienia: 1 misji

[1/1] Przetwarzam: https://warcraft.wiki.gg/wiki/A_Grave_Concern
OK - zapisano MISJA_ID = 408


In [2]:
parsuj_misje_z_url("https://warcraft.wiki.gg/wiki/A_Test_of_the_Hunt")

{'Źródło': {'url': 'https://warcraft.wiki.gg/wiki/A_Test_of_the_Hunt'},
 'Misje_EN': {'Podsumowanie_EN': {'Tytuł': 'A Test of the Hunt',
   'Start_NPC': 'Ranger Hannovia',
   'Koniec_NPC': 'Ranger Hannovia',
   'Wymagany_Poziom': '80-83',
   'Doświadczenie': '11,750',
   'Nagrody': '23 40'},
  'Cele_EN': {'Główny': {1: 'Gather 8 Invasive Lynx Pelts in the Sunstrider Isle.'},
   'Podrzędny': {1: 'Invasive Lynx Pelt\xa0(8)'}},
  'Treść_EN': {1: "So, you want a Farstrider recommendation? We don't hand those out like festival ribbons.",
   2: "If you want want[sic] to earn it, then you'll need to do what every other recruit does. Bring me lynx pelts, clean and proper.",
   3: "No shortcuts, no excuses. Let's see if you've got the spine for it."},
  'Postęp_EN': {1: 'Have you gotten those pelts yet or are you quitting already?'},
  'Zakończenie_EN': {1: "Well now, look at that. Clean cuts, no mess... maybe the kid's got more fire than I thought.",
   2: "Here, take this recommendation. Tell